In [ ]:
import requests
import json
from notebooks.secrets import access_token

base_url = 'https://api.vk.com/method/'


def execute(method, params):
    url = base_url + method + '?'
    for k, v in params.items():
        url += k + '=' + str(v) + '&'
    url += 'access_token=' + access_token + '&v=5.103'
    r = requests.get(url).text
    response = None
    err = None
    try:
        response = json.loads(r)
    except Exception as e:
        err = str(e)
    try:
        if 'response' not in response:
            err = response['error']
            err = str(err['error_code']) + ': ' + err['error_msg']
            response = None
        else:
            response = response['response']
    except:
        err = 'Error while parsing error'
    return err, response


error, result = execute('users.get', {'user_ids': 1})
print(error)
print(result)


ImportError: cannot import name 'access_token'

In [16]:
import notebooks.secrets
notebooks.secrets

ModuleNotFoundError: No module named 'notebooks'

In [ ]:
"from tqdm.notebook import tqdm\n",
    "\n",
    "user_ids = []\n",
    "for i in tqdm(range(3)):\n",
    "    err, conv = execute('messages.getConversations', {'count': 200, 'offset': i * 200})\n",
    "    if err is None:\n",
    "        conv_list = conv['items']\n",
    "        if len(conv_list) == 0:\n",
    "            break\n",
    "        for con in conv_list:\n",
    "            user_ids.append(con['conversation']['peer']['id'])\n",
    "        \n",
    "len(user_ids)"